In [1]:
# !pip install scikit-surprise
# !conda install -c conda-forge scikit-surprise

In [2]:
import pandas as pd
import re
import numpy as np
import random
from scipy.stats import uniform as sp_rand
from scipy.stats import randint
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 200)


In [3]:
from surprise import Reader
from surprise import SVD, SVDpp, NMF
from surprise import KNNBasic, KNNBaseline, KNNWithZScore, KNNWithMeans
from surprise import accuracy
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate, GridSearchCV, RandomizedSearchCV

In [4]:
my_seed = 55
random.seed(my_seed)
np.random.seed(my_seed)

In [5]:
df_users = pd.read_csv("../..//data/processed/users.csv")
df_items = pd.read_csv("../..//data/processed/items.csv")
df_items.item_name_lower = df_items.item_name_lower.map(lambda x : x[:25])
df_items.columns

Index(['user_id', 'brand_id', 'store_id', 'product_id', 'item_name_lower',
       'product_category_id', 'paid_price', 'size', 'on_sale', 'part_of_order',
       'top_brand', 'color', 'Category Name', 'price_bin'],
      dtype='object')

In [6]:
## Unique products
products =df_items.copy()
products.columns
products.drop(["user_id","store_id","paid_price","part_of_order","top_brand","color", "Category Name"],axis=1, inplace=True)
products.drop_duplicates(subset="product_id", inplace=True)
products.head()

,brand_id,product_id,item_name_lower,product_category_id,size,on_sale,price_bin
0,loft,62733a41,petite textured pencil pa,123 - Pants,NaN,False,3
1,loft,7ca9f965,blurred fairisle sweater,114 - Knits,NaN,False,3
2,loft,6273435d,lou grey eyelash sweater,114 - Knits,NaN,False,3
3,loft,62732b46,petite plaid pencil pants,123 - Pants,NaN,False,3
4,loft,627342fa,petite custom stretch tro,123 - Pants,NaN,False,3


In [7]:
"""
This function creates a sparse matrix and a simple group by for user - product combinations
When store_cat is set to True, it uses product category and store as proxy for product 
Note : change this to create sparse matrices instead of returning pandas dataframes
"""
def get_user_prod_matrix(df, store_cat = True):
    df_items["store_cat"] = df_items.apply(lambda x : x["store_id"] + " - "+ str(x["product_category_id"]), axis=1)
    
    if store_cat:
        sparse = pd.pivot_table(df_items, index="user_id", columns="store_cat", values="product_id", aggfunc="count")
    else:
        sparse = pd.crosstab(index = df_items["user_id"], columns = df_items["product_id"], values="product_id", aggfunc="count")
    
    grp = sparse.stack().dropna().reset_index()       
    grp = grp.rename(columns={0:"rating"} )

    if store_cat:
        grp = grp.rename(columns = {"store_cat": "product_id"})


    return (grp, sparse)

In [8]:
x1, y1 = get_user_prod_matrix(df_items)
print(x1.head())
x2, y2 = get_user_prod_matrix(df_items, False)
print(x2.head())
unique_user_sample = x2.user_id.sample(5).values.tolist()
print(unique_user_sample)


         user_id                    product_id  rating
0  1485369350003   dsw - 210 - Boots & Booties     2.0
1  1485369350003              hm - 124 - Jeans     1.0
2  1485369350003  jcrewfactory - 111 - Blouses     1.0
3  1485369350003    jcrewfactory - 114 - Knits     1.0
4  1485369350003          loft - 111 - Blouses     5.0
         user_id product_id  rating
0  1485369350003   00799605     1.0
1  1485369350003   01c00a9f     1.0
2  1485369350003   1389d2dd     1.0
3  1485369350003   1ff87a10     1.0
4  1485369350003   23583555     1.0
[1534200663687, 1540229647916, 1530899668174, 1495117820151, 1506291233259]


In [9]:
x1["base"] = "store_cat"
x2["base"] = "product"

full_ratings = x1.append(x2).reset_index(drop=True)
full_ratings.to_csv("../..//data/processed/all_ratings.csv",index=False)
x1.drop("base", axis=1, inplace=True)
x2.drop("base", axis=1, inplace=True)

In [10]:
print(full_ratings.head())
print(full_ratings.tail())

         user_id                    product_id  rating       base
0  1485369350003   dsw - 210 - Boots & Booties     2.0  store_cat
1  1485369350003              hm - 124 - Jeans     1.0  store_cat
2  1485369350003  jcrewfactory - 111 - Blouses     1.0  store_cat
3  1485369350003    jcrewfactory - 114 - Knits     1.0  store_cat
4  1485369350003          loft - 111 - Blouses     5.0  store_cat
             user_id product_id  rating     base
10336  1550516903728   94528138     1.0  product
10337  1550516903728   94944126     1.0  product
10338  1550516903728   98239982     1.0  product
10339  1550516903728   b8d0f825     1.0  product
10340  1550516903728   e4abcc38     1.0  product


In [11]:
def get_user_products(uid, df, n_count=10):
    cols_tokeep = ["product_id", "brand_id", "product_category_id", "rating","price_bin", "item_name_lower"]   
    col_names = ["Product ID", "Brand", "Category", "Times Bought","Price Bin", "Descr"]   


    user_products = df[df.user_id==uid]
    count = min(user_products.shape[0], n_count)
    user_top_products = user_products.sort_values("rating",ascending=False).head(count).merge(products, on="product_id")[cols_tokeep] 
    user_top_products.columns = col_names
    return user_top_products

In [12]:
def get_user_storecats(uid, df, n_count=10):
    cols_tokeep = ["product_id", "rating"]   
    col_names = ["Store - Category", "Times Bought"]   


    user_products = df[df.user_id==uid]
    count = min(user_products.shape[0], n_count)
    user_top_products = user_products.sort_values("rating",ascending=False).head(count)[cols_tokeep]
    user_top_products.columns = col_names
    return user_top_products

In [13]:
def get_n_recommendations(user_id, scores, n_recommendations=7):
    scores = scores[scores.uid == user_id]
    count = min(scores.shape[0], n_recommendations)
    cols_tokeep = ["est", "product_id", "brand_id", "product_category_id", "price_bin", "item_name_lower" ]
    col_names = ["Rating Estimate", "Product ID", "Brand", "Category", "Price Bin", "Descr"]
    temp_df = scores.sort_values("est", ascending=False).head(count).rename(columns={"iid":"product_id"}).merge(products, on="product_id")[cols_tokeep]
    temp_df.columns = col_names
    return temp_df

In [14]:
def get_n_recommendations_cat(user_id, scores, n_recommendations=7):
    scores = scores[scores.uid == user_id]
    count = min(scores.shape[0], n_recommendations)
    cols_tokeep = ["est", "iid"]
    col_names = ["Rating Estimate", "Store - Category"]
    temp_df = scores.sort_values("est", ascending=False).head(count)[cols_tokeep]
    temp_df.columns = col_names
    return temp_df

In [15]:
reader = Reader()
data1 = Dataset.load_from_df(x1,reader)
data2 = Dataset.load_from_df(x2,reader)

import re

def trainSurpriseModel(data, base, algo="SVD", search_iteration = 2):

    KNN_based = [KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline]
    SVD_based = [SVD, SVDpp]
    
    if (algo in SVD_based):
        param_grid = {
            'n_factors': [5,10,15,20],
            'n_epochs': [10, 20, 40, 80], 
            'lr_all': np.linspace(1e-4,2e-1,200),
            'reg_all': np.linspace(0,1,100),
            'random_state':[35]
        }
    
    if (algo == NMF):
        param_grid = {
            'n_factors': [5,10,15,20],
            'n_epochs': [10, 20, 40, 80], 
            'random_state':[35]
        }

    if (algo in KNN_based):
        param_grid = {
            'k': [1,3,5,10,15,20,25,30,35,40,45,50],
            'sim_options': {
                'name': ['msd', 'cosine'],
                'min_support': [1,3,5,10,15,20,25,30,35,40,45,50],
                'user_based': [False, True]},
            }
       
    print(param_grid)
    gs = RandomizedSearchCV(algo, param_grid, measures=['rmse', 'mae'], cv=3, refit=True,n_jobs=-1, n_iter=search_iteration)
    gs.fit(data)

    # best RMSE score
    print(gs.best_score['rmse'])
    print(gs.best_score['mae'])

    # combination of parameters that gave the best RMSE score
    print(gs.best_params['rmse'])
    print(gs.best_params['mae'])

    train_set = data.build_full_trainset()
    all_set = train_set.build_testset()
    anti_set = train_set.build_anti_testset()
    all_scores = pd.DataFrame(gs.test(all_set))
    anti_scores = pd.DataFrame(gs.test(anti_set))
    
    all_scores["algorithm"] = algo.__name__
    anti_scores["algorithm"] = algo.__name__    
    
    all_scores["score_type"] = "known"
    anti_scores["score_type"] = "anti"        

    all_scores["base"] = base
    anti_scores["base"] = base        


    return (all_scores, anti_scores)

In [16]:
algorithms = [SVD, SVDpp, NMF, KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline]
algorithms = [SVD]


n_iterations = 2

for algorithm in algorithms:
    print(f"\n****** Algorithm : {algorithm.__name__} ******")
    
    print("With Store - Cat Data ")
    all_s1, anti_s1 = trainSurpriseModel(data1, base="store_cat", algo=algorithm, search_iteration= n_iterations)

    print("\nWith Product Data ")
    all_s2, anti_s2 = trainSurpriseModel(data2, base="product", algo=algorithm, search_iteration= n_iterations)
    all_scores = all_s1.append(anti_s1).append(all_s2).append(anti_s2)
    all_scores.drop("details", axis=1, inplace=True)
    file_name = "../../data/processed/algo/all_reco_scores_" + algorithm.__name__ + ".csv"
    all_scores.to_csv(file_name,index=False)



****** Algorithm : SVD ******
With Store - Cat Data 
{'n_factors': [5, 10, 15, 20], 'n_epochs': [10, 20, 40, 80], 'lr_all': array([1.00000000e-04, 1.10452261e-03, 2.10904523e-03, 3.11356784e-03,
       4.11809045e-03, 5.12261307e-03, 6.12713568e-03, 7.13165829e-03,
       8.13618090e-03, 9.14070352e-03, 1.01452261e-02, 1.11497487e-02,
       1.21542714e-02, 1.31587940e-02, 1.41633166e-02, 1.51678392e-02,
       1.61723618e-02, 1.71768844e-02, 1.81814070e-02, 1.91859296e-02,
       2.01904523e-02, 2.11949749e-02, 2.21994975e-02, 2.32040201e-02,
       2.42085427e-02, 2.52130653e-02, 2.62175879e-02, 2.72221106e-02,
       2.82266332e-02, 2.92311558e-02, 3.02356784e-02, 3.12402010e-02,
       3.22447236e-02, 3.32492462e-02, 3.42537688e-02, 3.52582915e-02,
       3.62628141e-02, 3.72673367e-02, 3.82718593e-02, 3.92763819e-02,
       4.02809045e-02, 4.12854271e-02, 4.22899497e-02, 4.32944724e-02,
       4.42989950e-02, 4.53035176e-02, 4.63080402e-02, 4.73125628e-02,
       4.83170854e-02, 

0.14468968176095953
0.0340265965534387
{'n_factors': 10, 'n_epochs': 10, 'lr_all': 0.0151678391959799, 'reg_all': 0.8181818181818182, 'random_state': 35}
{'n_factors': 20, 'n_epochs': 20, 'lr_all': 0.08950251256281408, 'reg_all': 0.797979797979798, 'random_state': 35}


In [18]:
mypath = "../../data/processed/algo/"
from os import walk

all_scores = pd.DataFrame()
for (dirpath, dirnames, filenames) in walk(mypath):
    for filename in filenames:
        print(f"reading... {filename} ")
        csv_file_name  = mypath + filename
        df = pd.read_csv(csv_file_name)
        print(f"rows read : {df.shape[0]}")
        all_scores = all_scores.append(df)
        print(f"Combined rows so far : {all_scores.shape[0]}")

file_name = "../../data/processed/all_reco_scores.csv"
all_scores.to_csv(file_name,index=False)
all_scores = None


reading... all_reco_scores_SVD.csv 
rows read : 793941
Combined rows so far : 793941
reading... all_reco_scores_KNNWithMeans.csv 
rows read : 793941
Combined rows so far : 1587882
reading... all_reco_scores_KNNBasic.csv 
rows read : 793941
Combined rows so far : 2381823
reading... all_reco_scores_KNNWithZScore.csv 
rows read : 793941
Combined rows so far : 3175764
reading... all_reco_scores_KNNBaseline.csv 
rows read : 793941
Combined rows so far : 3969705
reading... all_reco_scores_SVDpp.csv 
rows read : 793941
Combined rows so far : 4763646
reading... all_reco_scores_NMF.csv 
rows read : 793941
Combined rows so far : 5557587


In [20]:
y1.head()

store_cat,6pmcom - 111 - Blouses,6pmcom - 114 - Knits,6pmcom - 141 - Casual,6pmcom - 143 - Cocktail,6pmcom - 152 - Jackets,6pmcom - 210 - Boots & Booties,6pmcom - 220 - Flats,6pmcom - 230 - Sandals,6pmcom - 240 - Heels,6pmcom - 320 - Shoulder,abercrombiefitch - 111 - Blouses,abercrombiefitch - 112 - T Shirts,abercrombiefitch - 113 - Tanks,abercrombiefitch - 114 - Knits,abercrombiefitch - 122 - Shorts,abercrombiefitch - 123 - Pants,abercrombiefitch - 124 - Jeans,abercrombiefitch - 132 - Long,abercrombiefitch - 141 - Casual,abercrombiefitch - 152 - Jackets,abercrombiefitch - 155 - Hoodies,abercrombiefitch - 161 - Tops,abercrombiefitch - 171 - Swimsuits,abercrombiefitch - 230 - Sandals,abercrombiefitch - 310 - Clutches,abercrombiefitch - 430 - Scarves,abercrombiefitch - 440 - Hats,adidas - 160 - Activewear,aeropostale - 112 - T Shirts,aeropostale - 122 - Shorts,aeropostale - 161 - Tops,aldo - 200 - Shoes,alexandani - 114 - Knits,alexandani - 410 - Jewelry,aloyoga - 160 - Activewear,americanapparel - 111 - Blouses,americanapparel - 112 - T Shirts,americanapparel - 121 - Skirts,americanapparel - 122 - Shorts,americanapparel - 124 - Jeans,americanapparel - 141 - Casual,americanapparel - 152 - Jackets,americanapparel - 155 - Hoodies,americanapparel - 163 - Bottoms,americanapparel - 171 - Swimsuits,americanapparel - 240 - Heels,americaneagleoutfitters - 111 - Blouses,americaneagleoutfitters - 112 - T Shirts,americaneagleoutfitters - 113 - Tanks,americaneagleoutfitters - 114 - Knits,americaneagleoutfitters - 121 - Skirts,americaneagleoutfitters - 122 - Shorts,americaneagleoutfitters - 123 - Pants,americaneagleoutfitters - 124 - Jeans,americaneagleoutfitters - 125 - Leggings,americaneagleoutfitters - 131 - Short,americaneagleoutfitters - 141 - Casual,americaneagleoutfitters - 152 - Jackets,americaneagleoutfitters - 154 - Vests,americaneagleoutfitters - 155 - Hoodies,americaneagleoutfitters - 161 - Tops,americaneagleoutfitters - 163 - Bottoms,americaneagleoutfitters - 171 - Swimsuits,americaneagleoutfitters - 210 - Boots & Booties,americaneagleoutfitters - 230 - Sandals,americaneagleoutfitters - 320 - Shoulder,americaneagleoutfitters - 410 - Jewelry,americaneagleoutfitters - 430 - Scarves,anntaylor - 110 - Tops,anntaylor - 111 - Blouses,anntaylor - 112 - T Shirts,anntaylor - 113 - Tanks,anntaylor - 114 - Knits,anntaylor - 121 - Skirts,anntaylor - 123 - Pants,anntaylor - 131 - Short,anntaylor - 141 - Casual,anntaylor - 143 - Cocktail,anntaylor - 152 - Jackets,anntaylor - 230 - Sandals,anntaylor - 240 - Heels,anntaylor - 350 - Totes,anntaylor - 460 - Belts,anthropologie - 111 - Blouses,anthropologie - 112 - T Shirts,anthropologie - 113 - Tanks,anthropologie - 114 - Knits,anthropologie - 121 - Skirts,anthropologie - 122 - Shorts,anthropologie - 123 - Pants,anthropologie - 131 - Short,anthropologie - 132 - Long,anthropologie - 141 - Casual,anthropologie - 144 - Formal,anthropologie - 152 - Jackets,anthropologie - 153 - Blazers,anthropologie - 154 - Vests,anthropologie - 155 - Hoodies,anthropologie - 163 - Bottoms,anthropologie - 171 - Swimsuits,anthropologie - 210 - Boots & Booties,anthropologie - 240 - Heels,anthropologie - 310 - Clutches,anthropologie - 320 - Shoulder,anthropologie - 410 - Jewelry,anthropologie - 420 - Sunglasses,anthropologie - 430 - Scarves,anthropologie - 460 - Belts,anthropologie - 470 - Other,aritzia - 112 - T Shirts,aritzia - 113 - Tanks,aritzia - 114 - Knits,aritzia - 123 - Pants,asos - 111 - Blouses,asos - 112 - T Shirts,asos - 113 - Tanks,asos - 114 - Knits,asos - 121 - Skirts,asos - 122 - Shorts,asos - 123 - Pants,asos - 124 - Jeans,asos - 125 - Leggings,asos - 131 - Short,asos - 132 - Long,asos - 141 - Casual,asos - 151 - Coats,asos - 152 - Jackets,asos - 153 - Blazers,asos - 154 - Vests,asos - 163 - Bottoms,asos - 171 - Swimsuits,asos - 172 - Coverups,asos - 210 - Boots & Booties,asos - 220 - Flats,asos - 230 - Sandals,asos - 240 - Heels,asos - 250 - Sneakers,asos - 320 - Shoulder,asos - 410 - Jewelry,asos - 42

In [50]:
from lightfm import LightFM
from scipy.sparse import csr_matrix, coo_matrix
from lightfm.evaluation import precision_at_k, recall_at_k

In [28]:
y_lightfm = coo_matrix(y1.fillna(0).values, dtype = "float32")



In [29]:
y_lightfm.get_shape

<bound method spmatrix.get_shape of <93x1385 sparse matrix of type '<class 'numpy.float32'>'
	with 3142 stored elements in COOrdinate format>>

In [30]:
model = LightFM(no_components=10, loss= "warp")

In [31]:
model.fit(y_lightfm, epochs=20)

In [52]:
print("Train precision: %.2f" % precision_at_k(model, y_lightfm, k=5).mean())
print("Train recall: %.2f" % recall_at_k(model, y_lightfm, k=5).mean())

Train precision: 0.54
Train recall: 0.18


In [32]:
dir(model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_finite',
 '_check_initialized',
 '_check_input_finite',
 '_check_test_train_intersections',
 '_construct_feature_matrices',
 '_get_lightfm_data',
 '_get_positives_lookup_matrix',
 '_initialize',
 '_process_sample_weight',
 '_reset_state',
 '_run_epoch',
 '_to_cython_dtype',
 'epsilon',
 'fit',
 'fit_partial',
 'get_item_representations',
 'get_params',
 'get_user_representations',
 'item_alpha',
 'item_bias_gradients',
 'item_bias_momentum',
 'item_biases',
 'item_embedding_gradients',
 'item_embedding_momentum',
 'item_embeddings',
 'k',
 'learning_rate',
 'learning_schedule',
 'loss',
 'max_sampled',
 'n',
 'no_

In [36]:
model.user_embeddings.shape

(93, 10)

In [59]:
# user_id = y1.iloc[0,0]

user_id = 1
y_lightfm.tocsr()[user_id].indices



array([  10,   92,  113,  140,  143,  163,  197,  198,  199,  200,  234,
        235,  236,  237,  238,  239,  242,  244,  246,  255,  256,  279,
        301,  305,  391,  392,  393,  394,  397,  399,  400,  403,  405,
        406,  414,  511,  517,  533,  552,  576,  636,  637,  638,  639,
        641,  643,  649,  651,  652,  658,  663,  664,  699,  705,  706,
        714,  715,  868,  869,  870,  871,  874,  879,  881,  886,  889,
        890,  892,  893,  894,  895,  897,  898,  902,  904,  913,  914,
        915,  916,  921,  924,  927,  930,  936,  939,  940,  947,  953,
        957,  962,  963, 1122, 1125, 1129, 1141, 1156, 1240, 1289, 1290,
       1291, 1292, 1295, 1296, 1298, 1299, 1300, 1303, 1304, 1305, 1308,
       1309, 1312, 1318, 1324, 1326, 1338, 1364, 1374], dtype=int32)

In [61]:
y_lightfm.tocsr()[user_id,10]

1.0

In [75]:
cols = set(range(y_lightfm.shape[1]))
cols = cols.difference(y_lightfm.tocsr()[user_id].indices)
cols

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 141,
 142,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190

In [57]:
y_lightfm.getrow(user_id).nonzero()

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 array([1374, 1364, 1338, 1326, 1324, 1318, 1312, 1309, 1308, 1305, 1304,
        1303, 1300, 1299, 1298, 1296, 1295, 1292, 1291, 1290, 1289, 1240,
        1156, 1141, 1129, 1125, 1122,  963,  962,  957,  953,  947,  940,
         939,  936,  930,  927,  924,  921,  916,  915,  914,  913,  904,
         902,  898,  897,  895,  894,  893,  892,  890,  889,  886,  881,
         879,  874,  871,  870,  869,  868,  715,  714,  706,  705,  699,
         664,  663,  658,  652,  651,  649,  643,  641,  639,  638,  637,
         636,  576,  552,  533,  517,  511,  414,  406,  405,  40

In [67]:
len(y_lightfm.tocsr().indices)

3142

In [70]:
print(y_lightfm)

  (0, 254)	2.0
  (0, 463)	1.0
  (0, 542)	1.0
  (0, 545)	1.0
  (0, 636)	5.0
  (0, 637)	3.0
  (0, 639)	15.0
  (0, 642)	6.0
  (0, 643)	14.0
  (0, 646)	6.0
  (0, 650)	2.0
  (0, 653)	2.0
  (0, 658)	2.0
  (0, 662)	1.0
  (0, 741)	9.0
  (0, 761)	1.0
  (0, 913)	3.0
  (0, 921)	1.0
  (0, 930)	2.0
  (0, 932)	4.0
  (0, 1022)	1.0
  (0, 1289)	1.0
  (0, 1299)	1.0
  (0, 1304)	3.0
  (0, 1350)	4.0
  :	:
  (90, 909)	1.0
  (90, 910)	1.0
  (90, 911)	2.0
  (91, 243)	1.0
  (91, 1289)	2.0
  (91, 1291)	1.0
  (91, 1292)	2.0
  (91, 1296)	1.0
  (91, 1303)	1.0
  (91, 1304)	5.0
  (92, 254)	1.0
  (92, 318)	5.0
  (92, 319)	1.0
  (92, 320)	1.0
  (92, 321)	3.0
  (92, 322)	2.0
  (92, 329)	1.0
  (92, 331)	1.0
  (92, 332)	4.0
  (92, 352)	1.0
  (92, 360)	1.0
  (92, 919)	1.0
  (92, 941)	1.0
  (92, 1289)	1.0
  (92, 1332)	1.0


In [41]:
y1

store_cat,6pmcom - 111 - Blouses,6pmcom - 114 - Knits,6pmcom - 141 - Casual,6pmcom - 143 - Cocktail,6pmcom - 152 - Jackets,6pmcom - 210 - Boots & Booties,6pmcom - 220 - Flats,6pmcom - 230 - Sandals,6pmcom - 240 - Heels,6pmcom - 320 - Shoulder,abercrombiefitch - 111 - Blouses,abercrombiefitch - 112 - T Shirts,abercrombiefitch - 113 - Tanks,abercrombiefitch - 114 - Knits,abercrombiefitch - 122 - Shorts,abercrombiefitch - 123 - Pants,abercrombiefitch - 124 - Jeans,abercrombiefitch - 132 - Long,abercrombiefitch - 141 - Casual,abercrombiefitch - 152 - Jackets,abercrombiefitch - 155 - Hoodies,abercrombiefitch - 161 - Tops,abercrombiefitch - 171 - Swimsuits,abercrombiefitch - 230 - Sandals,abercrombiefitch - 310 - Clutches,abercrombiefitch - 430 - Scarves,abercrombiefitch - 440 - Hats,adidas - 160 - Activewear,aeropostale - 112 - T Shirts,aeropostale - 122 - Shorts,aeropostale - 161 - Tops,aldo - 200 - Shoes,alexandani - 114 - Knits,alexandani - 410 - Jewelry,aloyoga - 160 - Activewear,americanapparel - 111 - Blouses,americanapparel - 112 - T Shirts,americanapparel - 121 - Skirts,americanapparel - 122 - Shorts,americanapparel - 124 - Jeans,americanapparel - 141 - Casual,americanapparel - 152 - Jackets,americanapparel - 155 - Hoodies,americanapparel - 163 - Bottoms,americanapparel - 171 - Swimsuits,americanapparel - 240 - Heels,americaneagleoutfitters - 111 - Blouses,americaneagleoutfitters - 112 - T Shirts,americaneagleoutfitters - 113 - Tanks,americaneagleoutfitters - 114 - Knits,americaneagleoutfitters - 121 - Skirts,americaneagleoutfitters - 122 - Shorts,americaneagleoutfitters - 123 - Pants,americaneagleoutfitters - 124 - Jeans,americaneagleoutfitters - 125 - Leggings,americaneagleoutfitters - 131 - Short,americaneagleoutfitters - 141 - Casual,americaneagleoutfitters - 152 - Jackets,americaneagleoutfitters - 154 - Vests,americaneagleoutfitters - 155 - Hoodies,americaneagleoutfitters - 161 - Tops,americaneagleoutfitters - 163 - Bottoms,americaneagleoutfitters - 171 - Swimsuits,americaneagleoutfitters - 210 - Boots & Booties,americaneagleoutfitters - 230 - Sandals,americaneagleoutfitters - 320 - Shoulder,americaneagleoutfitters - 410 - Jewelry,americaneagleoutfitters - 430 - Scarves,anntaylor - 110 - Tops,anntaylor - 111 - Blouses,anntaylor - 112 - T Shirts,anntaylor - 113 - Tanks,anntaylor - 114 - Knits,anntaylor - 121 - Skirts,anntaylor - 123 - Pants,anntaylor - 131 - Short,anntaylor - 141 - Casual,anntaylor - 143 - Cocktail,anntaylor - 152 - Jackets,anntaylor - 230 - Sandals,anntaylor - 240 - Heels,anntaylor - 350 - Totes,anntaylor - 460 - Belts,anthropologie - 111 - Blouses,anthropologie - 112 - T Shirts,anthropologie - 113 - Tanks,anthropologie - 114 - Knits,anthropologie - 121 - Skirts,anthropologie - 122 - Shorts,anthropologie - 123 - Pants,anthropologie - 131 - Short,anthropologie - 132 - Long,anthropologie - 141 - Casual,anthropologie - 144 - Formal,anthropologie - 152 - Jackets,anthropologie - 153 - Blazers,anthropologie - 154 - Vests,anthropologie - 155 - Hoodies,anthropologie - 163 - Bottoms,anthropologie - 171 - Swimsuits,anthropologie - 210 - Boots & Booties,anthropologie - 240 - Heels,anthropologie - 310 - Clutches,anthropologie - 320 - Shoulder,anthropologie - 410 - Jewelry,anthropologie - 420 - Sunglasses,anthropologie - 430 - Scarves,anthropologie - 460 - Belts,anthropologie - 470 - Other,aritzia - 112 - T Shirts,aritzia - 113 - Tanks,aritzia - 114 - Knits,aritzia - 123 - Pants,asos - 111 - Blouses,asos - 112 - T Shirts,asos - 113 - Tanks,asos - 114 - Knits,asos - 121 - Skirts,asos - 122 - Shorts,asos - 123 - Pants,asos - 124 - Jeans,asos - 125 - Leggings,asos - 131 - Short,asos - 132 - Long,asos - 141 - Casual,asos - 151 - Coats,asos - 152 - Jackets,asos - 153 - Blazers,asos - 154 - Vests,asos - 163 - Bottoms,asos - 171 - Swimsuits,asos - 172 - Coverups,asos - 210 - Boots & Booties,asos - 220 - Flats,asos - 230 - Sandals,asos - 240 - Heels,asos - 250 - Sneakers,asos - 320 - Shoulder,asos - 410 - Jewelry,asos - 42

In [44]:
df_users

,user_id,style_age_range_group,style_size_preference_none,style_size_preference_petite,style_size_preference_extra_long,style_size_preference_plus,style_size_preference_maternity,style_size_preference_skipped,style_looks_wanted_skipped,style_most_important_active,style_most_important_any,style_most_important_beach,style_most_important_dress,style_most_important_bags,style_most_important_jeans,style_most_important_jump,style_most_important_nothing,style_most_important_outwear,style_most_important_pants,style_most_important_shoes,style_most_important_tops,style_most_important_skipped,ariellecharnas,blaireadiebee,blakevond,chiaraferragni,hannahbronfman,jordynwoods,manrepeller,mayemusk,nicolettemason,seaofshoes,somethingnavy,weworewhat
0,1532487913121,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1539198476327,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1
2,1497894579289,5,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,1539745341992,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,1538675291370,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0
5,1524543123801,5,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
6,1529614244802,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
7,1549326478074,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0
8,1531437626855,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
9,1531280337633,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0


In [47]:
for i in df_users.columns[1:]:
    print(df_users[i].value_counts())

0    35
5    27
1    27
2    23
3     6
6     1
4     1
Name: style_age_range_group, dtype: int64
0    120
Name: style_size_preference_none, dtype: int64
0    95
1    25
Name: style_size_preference_petite, dtype: int64
0    117
1      3
Name: style_size_preference_extra_long, dtype: int64
0    116
1      4
Name: style_size_preference_plus, dtype: int64
0    119
1      1
Name: style_size_preference_maternity, dtype: int64
0    93
1    27
Name: style_size_preference_skipped, dtype: int64
0    88
1    32
Name: style_looks_wanted_skipped, dtype: int64
0    113
1      7
Name: style_most_important_active, dtype: int64
0    90
1    30
Name: style_most_important_any, dtype: int64
0    117
1      3
Name: style_most_important_beach, dtype: int64
0    89
1    31
Name: style_most_important_dress, dtype: int64
0    118
1      2
Name: style_most_important_bags, dtype: int64
0    109
1     11
Name: style_most_important_jeans, dtype: int64
0    114
1      6
Name: style_most_important_jump, dtype: int6